In [7]:
import pandas as pd
import numpy as np
import datetime

# Weather data

In [27]:
data_frame = pd.read_csv("MATF_Hackathon_2021/BelgradeAirport_2021-2012.csv")

clean_data = pd.DataFrame()
data_frame.head()

,DateTime,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E.,sss
0,2021-03-31 23:00,11.9,1010.5,1022.5,-0.5,67,Wind blowing from the north-northwest,1.0,NaN,NaN,...,NaN,NaN,10.0,5.8,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-31 22:00,12.9,1010.9,1022.7,NaN,62,Wind blowing from the north-northwest,1.0,NaN,NaN,...,NaN,NaN,10.0,5.8,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-31 21:00,14.3,1011.0,1022.9,NaN,57,Wind blowing from the north-northwest,1.0,NaN,NaN,...,NaN,NaN,10.0,5.9,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-31 20:00,15.3,1011.0,1022.9,-1.1,53,Wind blowing from the north-northwest,1.0,NaN,NaN,...,NaN,NaN,10.0,5.6,Trace of precipitation,12.0,NaN,NaN,NaN,NaN
4,2021-03-31 19:00,17.5,1011.1,1022.7,NaN,45,Wind blowing from the north-northeast,3.0,NaN,NaN,...,NaN,NaN,20.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN


### Parse DateTime

In [29]:
clean_data["year"] = data_frame["DateTime"].astype(str).str[:4].astype(int)
clean_data["month"] = data_frame["DateTime"].astype(str).str[5:7].astype(int)
clean_data["day"] = data_frame["DateTime"].astype(str).str[8:10].astype(int)
clean_data["hour"] = data_frame["DateTime"].astype(str).str[11:13].astype(int)
clean_data["min"] = data_frame["DateTime"].astype(str).str[14:].astype(int)